# Modeling on Ohio's Restaurant Yelp Review Data: Comparison Between Latent Dirichelet Allocation and Multinomial Logistic Regression

**Author:** Ningning Long, Yue You, Tian Xia

When running business like restaurants or cafes, owners of business care a lot of how they can make uses of customers’ 
reviews after they visit their businesses. The insights mined from customers’ reviews will help discover the weaknesses of the business and contribute to the improvements of service, product or ambience. This study is aimed for utilizing machine learning and natural language processing (NLP) techniques to analyze customers’ reviews from the [Yelp Dataset Challenge](https://www.yelp.com/dataset/challenge).

We focus on a subset data of the whole dataset challenge (i.e. the restaurants reviews in the state of Ohio). We are interested in using the latent dirichelet allocation (LDA) method to find out the topics underlying the customer’s reviews. The latent dirichelet allocation (LDA) is an unsupervised machine learning algorithm that can generate topics based on word frequency from a set of documents. We will use the topics to investigate how they match the reviews and predict the customers’ ratings. Meanwhile, in order to make a comparison, we will also compute the Tf-idf of reviews and use them to run the multinomial logistic regression of customers’ ratings. 

## Data Extraction & Cleaning

## Latent Dirichelet Allocation

## The Multinomial Logistic Regression

## Conclusion

## Author Contributions

This repository and project is the collaboration from **Ningning Long**, **Yue You** and **Tian Xia**. Their contributions to the project are summarized as:

**Ningning Long**:
-	‘*.ipynb’, latent dirichelet allocation modeling and analysis
-	‘makefile’
-	‘environment.yml’
-	Some write-up of ‘main.ipynb’

**Yue You**:
-	‘Modeling.ipynb’, statistical modeling of multinomial logistic regression
-	‘.gitignore’
-	Some write-up of ‘main.ipynb’

**Tian Xia**:
-	‘Data_Cleaning.ipynb’, data extraction and cleaning
-	‘README.md’
-	Some write-up of ‘main.ipynb’
-	‘LICENSE.md’
